In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
os.environ["CUDA_VISIBLE_DEVICES"]="0"; 

import pandas as pd
import numpy as np
from tqdm import *
pd.set_option('display.max_columns', None)

import random
import warnings
import math
import json
import scipy.stats as stats
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict
warnings.filterwarnings('ignore')

In [2]:
# # Results

# # Results From Paper Metrics

# In[2]:



import forest.benchmarking.distance_measures as dm
from scipy.spatial.distance import jensenshannon as jhn
def HellingerDistance(p, q):
    n = len(p)
    sum_ = 0.0
    for i in range(n):
        sum_ += (np.sqrt(p[i]) - np.sqrt(q[i]))**2
    result = (1.0 / np.sqrt(2.0)) * np.sqrt(sum_)
    return result


with open("Qiskit/Experiment 2 classical/results/saveRQ1.json","r") as file:
    RQ1C = json.load(file)

with open("Cirq/Experiment 2 classical/results/saveRQ1.json","r") as file:
    RQ1c = json.load(file)

with open("Rigetti/Experiment 2 classical/results/saveRQ1.json","r") as file:
    RQ1r = json.load(file)

for k in RQ1c.keys():
    RQ1C[k] = RQ1c[k]

for k in RQ1r.keys():
    RQ1C[k] = RQ1r[k]
    
with open("Q2RQ1.json","r") as file:
    nRQ1C = json.load(file)

with open("C2RQ1.json","r") as file:
    nRQ1c = json.load(file)

with open("R2RQ1.json","r") as file:
    nRQ1r = json.load(file)

for k in nRQ1c.keys():
    nRQ1C[k] = nRQ1c[k]

for k in nRQ1r.keys():
    nRQ1C[k] = nRQ1r[k]

In [3]:
another = {}
programs = list(RQ1C[list(RQ1C.keys())[0]].keys())
for program in programs:
    bk_hln  = []
    bk_hlf  = []
    for bk in RQ1C.keys():
        hln = np.mean(nRQ1C[bk][program]["NoiseHL"])
        hlf = np.mean(RQ1C[bk][program]["FilterHL"])
        bk_hln.append(hln)
        bk_hlf.append(hlf)
    another[program] = {"Avg Filter HL":np.mean(bk_hlf),
                       "Avg Noise HL":np.mean(bk_hln)}


# In[19]:


def sort_key(v):
    x = v.str.replace("P","")
    return x.astype(int)
programs = pd.DataFrame.from_dict(another).T
programs["Improv HL"] = np.round((programs["Avg Noise HL"].values-programs["Avg Filter HL"].values)/programs["Avg Noise HL"].values*100,2)
programs = programs.sort_index(key=sort_key)
programs.to_csv("results2/RQ1_Classical_HL_PP.csv")
programs
print("average improvement:",programs["Improv HL"].mean())

average improvement: -52.1594375


In [4]:


# In[21]:


# from Abstract_Interface import BackendFactory
# import benchmark_circuits as bc
# from util_imports import *
# backend_factory = BackendFactory()
# backend = backend_factory.initialize_backend()
# load_circuits()


# In[22]:


# for x in programs[programs["Improv HL"]<0].index:
#     circ = QuantumCircuit.from_qasm_file("random_circuits/{}.qasm".format(x))
# #     algo = get_circuit_class_object(x)
# #     inp = algo.get_full_inputs()[0]
# #     result = algo.get_result(backend,inp)
# #     print([(x["bin"],x["prob"]) for x in result["probability"]])
#     print("circuit name {}, depth {}".format(x,circ.depth()))
#     print(circ)
# #     break
# # algo.circ.decompose(reps=4).draw(output='mpl')


# In[23]:


another = {}

for bk in RQ1C.keys():
    bk_hln  = []
    bk_hlf  = []
    for program in RQ1C[bk].keys():
        hln = np.mean(nRQ1C[bk][program]["NoiseHL"])
        hlf = np.mean(RQ1C[bk][program]["FilterHL"])
        bk_hln.append(hln)
        bk_hlf.append(hlf)
    another[bk.replace("Fake","")] = {"Avg Filter HL":np.mean(bk_hlf),
                       "Avg Noise HL":np.mean(bk_hln)}


# In[24]:


bk_perspective = pd.DataFrame.from_dict(another).T
bk_perspective["Improv HL"] = np.round((bk_perspective["Avg Noise HL"].values-bk_perspective["Avg Filter HL"].values)/bk_perspective["Avg Noise HL"].values*100,2)
bk_perspective.sort_index().round(2).to_csv("results2/RQ1_Classical_HL_BK.csv")
bk_perspective.sort_index().round(2)


# In[25]:


print("average improvement:",bk_perspective["Improv HL"].mean())

average improvement: -46.90038461538462
